<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200">
</div>

# Hybrid Compute for QEC workflows

Quantinuum has a hybrid compute capability for Quantum Error Correction (QEC) workflows. This capability executes [Web Assembly (WASM)](https://webassembly.org/) in the H-Series stack and enables classical computation during real-time execution of a quantum circuit. Unlocking experimentation of basic QEC workflows is an important milestone, as Quantinuum plans to upgrades its H-Series hardware and deploy practical QEC schemes at scale. Wasm has been used, previously, in quantum error correction schemes [[arxiv.2208.01863](https://arxiv.org/abs/2208.01863), [arxiv.2305.03828](https://arxiv.org/pdf/2305.03828)]. Syndrome measurement results are sent to a classical computer where a decoding algorithm is used to determine recovery operations and update quantum circuits in real-time. 

Wasm is a binary instruction format. To write in and compile programs to it, another language such as Rust or C/ C++ is used to write your desired classical functions and compile these to Wasm. The quantum computing workflow will then contain a call to Wasm. The folder this notebooks is in contains examples of this workflow. Wasm is used because it is a fast, safe, expressive, and easy to compile to using familiar programming languages.

*Setup* goes over the setup instructions for setting up a python environment that can work with hybrid compute. The *Generating Wasm* section provides an overview on this compilation step. *Basic Usage* details using TKET capabilites to link the Wasm module to a TKET circuit as well as submission to H-Series. The *Repetition Code* section narrates how the Wasm feature can be used to enable a decoder-style lookup table in the Repetition code workflow.

*Note: Additional Resources* 

The Mid-circuit Measurement and Reset (MCMR) with Qubit Reuse features are used in the Repetition Code workflow, which is also showcased [here](https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_mid_circuit_measurement.ipynb). An explanation of the H-Series submission workflow is found [here](https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_circuit_submissions.ipynb). The Wasm module is imported from pytket as well as several optional conditional operators. More information on the conditional operations available can be found in the TKET user manual at [Classical and Conditional Operations](https://tket.quantinuum.com/api-docs/classical.html).


**Contents**
* [Setup and Workflow](#Setup-and-Workflow)
* [Basic Usage](#Basic-Usage)
* [Repetition Code](#Repetition-Code)
* [Summary](#Summary)


## Setup and Workflow


To create and use your own Hybrid Compute functions in your workflow, it is necessary to use Wasm and a language that compiles code to Wasm. These instructions contain the steps for setting up an environment to do this.

Wasm can be generated by composing Rust and C/ C++ code, followed by a compilation step with the Rust compiler or LLVM toolchain (Clang compiler). Once generated, the Wasm binary can be injected into the pytket workflow. The main benefit is to enable classical compute in the H-Series stack during real-time execution of the quantum circuit.

It is recommended to either use Rust or C++ for the compilation language to Wasm. The languages have functionality that easily compiles to Wasm. For other language options see [Wasm Getting Started Developer's page](https://webassembly.org/getting-started/developers-guide/).

<div style="text-align: center;">
    <img src="../figures/hybrid_compute_fig1.png" width="500">
</div>

The following subsections will go over:
1. Install either Rust or C/C++
1. Define the classical functions to use in your program
1. Compile to Wasm
1. Use Compiled Wasm

**Note:** This notebook uses existing Wasm examples provided in this folder's subdirectories. The subfolders are organized as follows.

---
    ├── Hybrid-compute              <- Examples of how to use Hybrid Compute via WASM
        |
        ├── repeat_until_success    <- Repeat Until Success source code
        ├── repetition_code         <- Repetition Code source code
---



### 1. Install Rust or C/C++


#### Rust

If you're new to Rust, see the [The Rust Programming Language](https://doc.rust-lang.org/book/title-page.html) book. Note that Cargo is the Rust package manager. For more info on Cargo, see [The Cargo Book](https://doc.rust-lang.org/cargo/guide/).

If using Visual Studio code, you can also install the `rust-analyzer` extension in the Extensions tab.


##### a. Install Rust

Install Rust using the [Rust Installer](https://www.rust-lang.org/tools/install).

**Windows Users:** Before running the installer, you'll need to install the [Microsoft C++ Build Tools](https://visualstudio.microsoft.com/visual-cpp-build-tools/).


##### b. Create a new Rust project

Navigate to wherever you plan to store your Rust project files, and run: `cargo new --lib rus_wasm`. The project name `rus_wasm` can be whatever project name you want to use.

This creates a new library in a subdirectory with the project name you used with everything you need to get going.

* The `/src/lib.rs` file contains some generated Rust code. This code can be erased and replaced with code you plan to use in your hybrid compute program.
* Inside this subdirectory is a file called `Cargo.toml`. This file will be used to configure your build.


#### C/ C++

For Windows 10/ 11 users, the Visual Studio build tools (also available through a standard Visual Studio installation) should be downloaded. During installation, the C++ Desktop Development group, Clang, and LLVM options should be selected.

For Ubuntu 20.04 users, the development tools group needs to be installed with Aptitude. The LLVM and Clang compilers need to be installed separately.

MacOS users must install XCode command line tools and also brew. The latest version of Clang can be installed with brew.


### 2. Define classical functions in your language of choice

Define the classical functions you plan to use in your hybrid compute program. These will be the classical functions that your quantum program will run during execution. In Rust, edit the `src/lib.rs` file. For C/C++, edit the `src/lib.c` file. 

Examples of the types of functions that can be put in Rust or C code are provided in the respective folders within each of the example folders. *We recommend looking at one of the provided examples to know which functions and attributes are required for Hybrid Compute to work.*

*Note:* The standard C (C++) library cannot be used in a program that is intended to be compiled to Wasm.

### 3. Compile code to Wasm

##### Rust 

1. Configure the `Cargo.toml` file. You can use one of the examples provided to modify yours. 
1. Navigate to your Rust project directory from the command line.
1. Run: `cargo build`. 
    - If successful, a `target` folder and `Cargo.lock` file will be created, alongside the output of the command displaying "Finished."
    - If failed, Cargo will output a messaging related to the reason.
1. Run: `rustup target add wasm32-unknown-unknown`. This allows Rust to used Wasm as a compilation target. This only needs to be run once for your environment. If you have run this previously when compiling Rust to Wasm, you do not need to run it again. 
1. Run: `cargo build --release --target wasm32-unknown-unknown`. This compiles your Rust code to Wasm.


##### C/C++

The `C`/ `C++` compilation uses the `LLVM` toolchain and requires that no standard headers be used within the program. The compilation can be invoked with the `Clang` (`Clang++`) compiler with specific options to compile `lib.c` (`lib.cpp`) into a Wasm binary, `lib.wasm`:

```clang --target=wasm32 --no-standard-headers -Wl,--no-entry -Wl,--export-all -o lib.wasm lib.c```

The following compiler options are used upon Clang invocation:

* `--target=wasm32`: Specify the target output as 32 bit Wasm binary.
* `--no-standard-headers`: A compiler option to prevent usage of `libc`.
* `-Wl,--no-entry-point`: A linker option that disables the check for a main function in the `c` scipt.

#### 4. Use compiled Wasm

In order to use the compiled wasm from your quantum program, you will want to find the associated `.wasm` file that the compilation step created. With Rust, this can be found in the `./target/wasm32-unknown-unknown/release` folder of your project. You'll note a file with the name of your project with the extension `.wasm`. To use this in your quantum program, you can copy-paste it into another folder, or ensure that the file path for your Wasm file handler points to this location.

#### Wasm Guidelines

Some limitations exist on what kinds of functions or capabilities are enabled. 

1. Guidelines writing source code:
    * The `*.rs` or `*.c` source must contain an empty void method named `init`. This is to initialize the Wasm environment.
    * Examples of the types of functions that can be put into code are provided in the `c/src` or `rust/src` folders within each of the example folders.
    * The function to compile to Wasm must accept multiple integers and return exactly one integer.
    * Rust source code can use the Rust standard library.
    * C source code cannot use the C standard library, WASI is needed for this, which breaks the Wasm memory sandbox.
    * The following is not allowed:
        * Random number generation
        * File I/O
        * Exception Handling
1. Quantum programs can only call Wasm functions that accept multiple integers and return an integer. 
    * For example, examine the call `x = wasm_func(a, b, c);` The `wasm_func` function can exist in QASM where `a`, `b`, `c`, and `x` are classical registers. 
    * Such function calls have timing restrictions. Wasm calls in quantum programs have been run that took multiple milliseconds.
1. The state of a Wasm program persists between calls in the quantum programs within the limits of a job chunk window (~300 shots). 
    * This means global variables can hold a mutable state that is modified between calls.
    * One can imagine using various global arrays. It's possible to have global pointers work with heap allocated data structures (for example using unsafe Rust features when compiling to Wasm).
1. Currently, the combined Quantum Web Assembly programs have to come in under 6 MB*.

## Basic Usage

To inject Wasm into a TKET program and enable processing on the H-Series devices or emulators, three components must be incorporated in the workflow.

1. [pytket.wasm.WasmFileHandler](https://tket.quantinuum.com/api-docs/wasm.html#pytket.wasm.WasmFileHandler): Enables addition of the Wasm module to the TKET program as well as provides capability to verify logic within the Wasm module.
1. [pytket.circuit.Circuit.add_wasm_to_reg](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.add_wasm_to_reg): Adds the linked Wasm to the TKET program as a classical operation.
1. [wasm_file_handler](https://tket.quantinuum.com/extensions/pytket-quantinuum/api.html#pytket.extensions.quantinuum.QuantinuumBackend.process_circuits) kwarg on `QuantinuumBackend.process_circuits`: The linked Wasm module must be specified during job submission to H-Series.

Components 1 and 2 are used in the `Job Construction` subsection. Component 3 is used in the `Job Submission` subsection. 

Usage of Wasm is showcased in a Repeat Until Success (RUS) example. The RUS protocol, initially introduced in [arxiv.1311.1074](https://arxiv.org/abs/1311.1074), is a non-deterministic decompostion technique to approximate a single-qubit unitary within an allowable error threshold. The RUS circuit is composed of multiple subcircuts and each subcircuit runs successively if a specific condition is satisfied.

### Circuit Construction

We illustrate an example calling Wasm from within a pytket `Circuit`.

A 2-qubit circuit is initialized with 4 Bit registers. 

* `creg0` with 1 qubit
* `creg1` with 1 qubit
* `cond` with 32 qubits
* `count` with 32 qubits

In [1]:
from pytket.circuit import Circuit, Qubit
from pytket.circuit.logic_exp import reg_lt

circuit = Circuit(2, name=f"RUS Circuit")

# Add classical registers
creg0 = circuit.add_c_register("creg0", 1)
creg1 = circuit.add_c_register("creg1", 1)
cond = circuit.add_c_register("cond", 32)
count = circuit.add_c_register("count", 32)

The `pytket.wasm.WasmFileHandler` instance is used to link to the Wasm module.

In [2]:
import pathlib
from pytket.wasm import WasmFileHandler

rus_dir = pathlib.Path().cwd().joinpath("repeat_until_success")
wasm_file = rus_dir.joinpath("rus.wasm")
wfh = WasmFileHandler(wasm_file)

print(repr(wfh))

Functions in wasm file with the uid f5265965c9f207c02e626e3fd48183b545f74877a27c62c3fcc2cc68bee5a36a:
function 'init' with 0 i32 parameter(s) and 0 i32 return value(s)
function 'add_count' with 2 i32 parameter(s) and 1 i32 return value(s)



The linked Wasm module is added to `pytket.circuit.Circuit` instance using `add_wasm_to_reg`. The Wasm method `add_count` is called and the bit registers `creg1` and `count` are supplied. The output of `add_count` is written to `cond`. All the operations in this block are conditional operations on the decimal value of the  bit register `cond`.

In [3]:
n_repetitions = 5
cond_execute = 3
circuit.add_c_setreg(0, cond)

for loop_iter in range(1, n_repetitions + 1):
    circuit.H(0, condition=reg_lt(cond, cond_execute))
    circuit.CX(0, 1, condition=reg_lt(cond, cond_execute))
    circuit.Measure(Qubit(1), creg1[0], condition=reg_lt(cond, cond_execute))

    # Call Wasm
    circuit.add_wasm_to_reg("add_count", wfh, [creg1, count], [cond], condition=reg_lt(cond, cond_execute))

    circuit.add_c_setreg(loop_iter, count, condition=reg_lt(cond, cond_execute))
    circuit.Reset(0, condition=reg_lt(cond, cond_execute))

circuit.Measure(Qubit(0), creg0[0])
circuit.Measure(Qubit(1), creg1[0])

### Job Submission

A `QuantinuumBackend` instance is instantiated with the backend target specified as an emulator, `H2-1E`.

In [5]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend("H2-1E")
backend.login()

c:\Users\Irfan.Khan\.conda\envs\pytket-quantinuum-env\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


The RUS circuit is compiled to the H-Series gate set.

In [6]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)

In [7]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circuit)

The RUS circuit is submitted to the H-Series emulator using `process_circuits`. The `wasm_file_handler` keyword argument is used to submit the linked Wasm module to H-Series as well.

In [ ]:
handle = backend.process_circuit(compiled_circuit, n_shots=100, wasm_file_handler=wfh)

The status of the circuit can be queried with the `circuit_status` method.

In [9]:
status = backend.circuit_status(handle)
print(status)
print(status.message)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message='{"name": "RUS Circuit", "submit-date": "2024-06-04T13:45:00.754580", "result-date": null, "queue-position": null, "cost": null, "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)
{"name": "RUS Circuit", "submit-date": "2024-06-04T13:45:00.754580", "result-date": null, "queue-position": null, "cost": null, "error": null}


Once the job result is ready `get_result` can be used to retrieve the data.

In [10]:
result = backend.get_result(handle)

In [11]:
distribution = result.get_counts(creg1)
logical_fidelity = distribution.get((0,)) / sum(distribution.values())
print(logical_fidelity)

0.11


## Repetition Code

The classical operations on bits and bit registers using `pytket.circuit.logic_exp` can be replaced with Wasm.

In [19]:
from pathlib import Path
from pytket.wasm import WasmFileHandler

wasm_file_path = Path().cwd().joinpath("repetition_code") / "rust" / "rc.wasm"
wasm_file_handler = WasmFileHandler(wasm_file_path, check_file=True)

Classical expressions can be applied to classical bits using the `pytket.circuit.logic_exp` submodule. These expressions perform boolean operations on classical bit registers.

A pytket `Circuit` is constructed using the `QubitRegister` and `BitRegister` objects to allocate qubits and bits to the circuit. The circuit contains two quantum registers, `q` and `a`, and six classical registers:
* `c`
* `pfu`
* `pfu_new`
* `pfu_old`
* `syn`
* `syn_old`
* `syn_new`

In [20]:
from pytket.circuit import Circuit, QubitRegister, BitRegister

circuit = Circuit()
circuit.name = "repetition-code"

qreg = QubitRegister("q", 3)
circuit.add_q_register(qreg)

areg = QubitRegister("a", 1)
circuit.add_q_register(areg)

creg = BitRegister("c", 3)
circuit.add_c_register(creg)

pfu = BitRegister("pfu", 3)
circuit.add_c_register(pfu)

pfu_old = BitRegister("pfu_old", 3)
circuit.add_c_register(pfu_old)

syn = BitRegister("syn", 2)
circuit.add_c_register(syn)

syn_old = BitRegister("syn_old", 2)
circuit.add_c_register(syn_old)

syn_new = BitRegister("syn_new", 2)
circuit.add_c_register(syn_new)

BitRegister("syn_new", 2)

The cell below adds the necessary operations to measure the stabilizers $\hat{Z}_0 \hat{Z}_1$ and $\hat{Z}_1 \hat{Z}_2$.

Classical operations on the syndrome bit registers are applied. A barrier is added to ensure these classical operations are applied after the syndrome extraction above. 

`pytket.circuit.logic_exp.RegXor` is used to perform the `XOR` operation on register `syn_old` and `syn`. This is added to the circuit with the method `add_classicalexpbox_register`. Finally the state of a classical register, `syn`, is copied to another classical_register, `syn_old`. `syn_old[0]`, `syn_old[1]` and `syn_old[2]` are equal to `syn[0]`, `syn[1]` and `syn[2]`.

In [21]:
from pytket.circuit.logic_exp import RegXor

circuit.CX(qreg[0], areg[0])
circuit.CX(qreg[1], areg[0])
circuit.Measure(areg[0], syn[0])
circuit.Reset(areg[0])

circuit.CX(qreg[1], areg[0])
circuit.CX(qreg[2], areg[0])
circuit.Measure(areg[0], syn[1])
circuit.Reset(areg[0])

circuit.add_classicalexpbox_register(RegXor(syn_old, syn), syn_new)
circuit.add_c_copyreg(syn, syn_old)

[CX q[0], a[0]; CX q[1], a[0]; Measure a[0] --> syn[0]; Reset a[0]; CX q[1], a[0]; CX q[2], a[0]; Measure a[0] --> syn[1]; Reset a[0]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; CopyBits syn[0], syn[1], syn_old[0], syn_old[1]; ]

The `add_wasm_to_reg` method is used to apply the `decode3` function, defined in the Wasm module, to the bit registers, `syn` and `pfu`. The output of `decode3` is written to the bit register, `pfu_old`. `add_c_xor_to_registers` is used to set the value of `pfu_old` to the output of the expression, `pfu_old` ^ `pfu`. `add_c_setreg` is used to set the all the bits in the classical register, `pfu`, to `False`.

In [22]:
circuit.add_wasm_to_reg(
    "decode3", 
    wasm_file_handler, 
    [syn_new, pfu], 
    [pfu_old]
)

[CX q[0], a[0]; CX q[1], a[0]; Measure a[0] --> syn[0]; Reset a[0]; CX q[1], a[0]; CX q[2], a[0]; Measure a[0] --> syn[1]; Reset a[0]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; WASM syn_new[0], syn_new[1], pfu[0], pfu[1], pfu[2], pfu_old[0], pfu_old[1], pfu_old[2], _w[0]; CopyBits syn[0], syn[1], syn_old[0], syn_old[1]; ]

Three measurement operations are applied to qubits within `qreg`. The outcome of the measurements is stored in the corresponding bits within `creg`, i.e. a measurement on `qreg[0]` will have a measurement outcome stored in `creg[0]`. Three Measurement operations are added to each qubit in `qreg` and the corresponding bit in `creg`.

In [23]:
for i in range(3):
    circuit.Measure(qreg[i], creg[i])

`add_classicalexpbox_bit`, `add_c_setbits`, `add_classicalexpbox_register` are used to perform additional operations on the classical registers. The `^` operation is also used instead of `RegXOR` to perform `XOR` on two classical registers. `add_classicalexpbox_bit` is used to perform operations on specific bits, whilst `add_classicalexpbox_register` allows classical operations on entire bit registers.

In [24]:
circuit.add_classicalexpbox_bit(creg[0] ^ creg[1], [syn[0]])
circuit.add_classicalexpbox_bit(creg[1] ^ creg[2], [syn[1]])

[CX q[0], a[0]; CX q[1], a[0]; Measure q[0] --> c[0]; Measure a[0] --> syn[0]; Reset a[0]; CX q[1], a[0]; CX q[2], a[0]; Measure q[1] --> c[1]; Measure a[0] --> syn[1]; Measure q[2] --> c[2]; Reset a[0]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; WASM syn_new[0], syn_new[1], pfu[0], pfu[1], pfu[2], pfu_old[0], pfu_old[1], pfu_old[2], _w[0]; CopyBits syn[0], syn[1], syn_old[0], syn_old[1]; ClassicalExpBox c[0], c[1], syn[0]; ClassicalExpBox c[1], c[2], syn[1]; ]

In [25]:
circuit.add_classicalexpbox_register(syn_old ^ syn, syn_new)

[CX q[0], a[0]; CX q[1], a[0]; Measure q[0] --> c[0]; Measure a[0] --> syn[0]; Reset a[0]; CX q[1], a[0]; CX q[2], a[0]; Measure q[1] --> c[1]; Measure a[0] --> syn[1]; Measure q[2] --> c[2]; Reset a[0]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; WASM syn_new[0], syn_new[1], pfu[0], pfu[1], pfu[2], pfu_old[0], pfu_old[1], pfu_old[2], _w[0]; CopyBits syn[0], syn[1], syn_old[0], syn_old[1]; ClassicalExpBox c[0], c[1], syn[0]; ClassicalExpBox c[1], c[2], syn[1]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; ]

In [26]:
circuit.add_wasm_to_reg(
    "decode3", 
    wasm_file_handler, 
    [syn_new, pfu], 
    [pfu_old]
)

[CX q[0], a[0]; CX q[1], a[0]; Measure q[0] --> c[0]; Measure a[0] --> syn[0]; Reset a[0]; CX q[1], a[0]; CX q[2], a[0]; Measure q[1] --> c[1]; Measure a[0] --> syn[1]; Measure q[2] --> c[2]; Reset a[0]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; WASM syn_new[0], syn_new[1], pfu[0], pfu[1], pfu[2], pfu_old[0], pfu_old[1], pfu_old[2], _w[0]; CopyBits syn[0], syn[1], syn_old[0], syn_old[1]; ClassicalExpBox c[0], c[1], syn[0]; ClassicalExpBox c[1], c[2], syn[1]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; WASM syn_new[0], syn_new[1], pfu[0], pfu[1], pfu[2], pfu_old[0], pfu_old[1], pfu_old[2], _w[0]; ]

In [27]:
circuit.add_c_setreg(False, pfu)
circuit.add_classicalexpbox_register(pfu_old ^ creg, pfu_old)

[CX q[0], a[0]; CX q[1], a[0]; Measure q[0] --> c[0]; Measure a[0] --> syn[0]; Reset a[0]; CX q[1], a[0]; CX q[2], a[0]; Measure q[1] --> c[1]; Measure a[0] --> syn[1]; Measure q[2] --> c[2]; Reset a[0]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; WASM syn_new[0], syn_new[1], pfu[0], pfu[1], pfu[2], pfu_old[0], pfu_old[1], pfu_old[2], _w[0]; CopyBits syn[0], syn[1], syn_old[0], syn_old[1]; ClassicalExpBox c[0], c[1], syn[0]; ClassicalExpBox c[1], c[2], syn[1]; ClassicalExpBox syn[0], syn[1], syn_old[0], syn_old[1], syn_new[0], syn_new[1]; WASM syn_new[0], syn_new[1], pfu[0], pfu[1], pfu[2], pfu_old[0], pfu_old[1], pfu_old[2], _w[0]; SetBits(000) pfu[0], pfu[1], pfu[2]; ClassicalExpBox c[0], c[1], c[2], pfu_old[0], pfu_old[1], pfu_old[2]; ]

The repetition code circuit with the Wasm operation can be submitted to the `H2-1E` emulator using the usual workflow: compilation, costing, submission, status check and, finally, job retreival. The linked Wasm module must be specified during job submission.

In [28]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1E")
backend.login()

In [29]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)
render_circuit_jupyter(compiled_circuit)

In [30]:
handle = backend.process_circuit(compiled_circuit, n_shots=1000, wasm_file_handler=wasm_file_handler)

In [31]:
backend.circuit_status(handle)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message='{"name": "repetition-code", "submit-date": "2024-06-10T20:35:25.450406", "result-date": null, "queue-position": null, "cost": "24.8", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [32]:
result = backend.get_result(handle)

In [33]:
result.get_distribution(creg)

{(0, 0, 0): 0.985, (0, 0, 1): 0.004, (0, 1, 0): 0.007, (1, 0, 0): 0.004}

In [34]:
import numpy as np
import pandas as pd

from pytket.backends.backendresult import BackendResult
from pytket.circuit import BitRegister

def logical_error(
    result: BackendResult,
    n_shots: int,
    classical_register: BitRegister,
    pauli_frame_register: BitRegister,
) -> pd.DataFrame:
    raw0_tot = 0
    cor0_tot = 0
    raw1_tot = 0
    cor1_tot = 0

    c_dist = result.get_distribution(classical_register)
    pfu_dist = result.get_distribution(pauli_frame_register)

    for c, p in zip(c_dist.keys(), pfu_dist.keys()):
        nc = len(c)
        cr1 = sum(c)
        cr0 = nc - cr1
        if cr1 < cr0:
            raw0_tot += 1
        elif cr1 > cr0:
            raw1_tot += 1
        
        np = len(p)
        pr1 = sum(p)
        pr0 = np - pr1

        if pr1 < pr0:
            cor0_tot += 1
        elif pr1 > pr0:
            cor1_tot += 1

    raw0_tot = raw0_tot/n_shots
    raw1_tot = raw1_tot / n_shots
    cor0_tot = cor0_tot / n_shots
    cor1_tot = cor1_tot / n_shots

    error0 = (cor0_tot * (1 - cor0_tot)) / n_shots
    error1 = (cor1_tot * (1 - cor1_tot)) / n_shots

    return pd.DataFrame([
        ["Raw Zeros", raw0_tot],
        ["Raw Ones", raw1_tot],
        ["Corrected Zeros", cor0_tot],
        ["Corrected Ones", cor1_tot],
        ["Error Corrected Zeros", error0],
        ["Error Corrected Ones", error1]
    ])

logical_error(result, 100, creg, pfu_old)

,0,1
0,Raw Zeros,0.040000
1,Raw Ones,0.000000
2,Corrected Zeros,0.030000
3,Corrected Ones,0.010000
4,Error Corrected Zeros,0.000291
5,Error Corrected Ones,0.000099


## Summary

The hybrid compute feature enables classical compute logic compiled to a Wasm binary to be linked and added to a TKET program for processing on H-Series. The classical compute portion of the program is processed during real-time execution of the quantum ciruit. This is the first major step to practical QEC at scale as H-Series hardware evolves towards fault-tolerance. The 3-qubit repetition code protocol uses Wasm for a look-up table to enable decoding of error syndromes.

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>